In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf


In [13]:
df = pd.read_csv('congress_main.csv')
df.head()
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['TransactionDate'] = df['TransactionDate'].dt.strftime('%Y-%m-%d')
df.Ticker = df.Ticker.str.lower()
df.head()

,ReportDate,TransactionDate,Ticker,Representative,Transaction,Amount,Party,House,Range,Sentiment,Price,Low_End,High_End,Adjusted_Low_End,Adjusted_High_End,Running_Portfolio_Low_End,Running_Portfolio_High_End,Est_Portfolio_Value,Shares_High_End,Shares_Low_End
0,2018-05-15,2018-03-05,wfc,A. Mitchell Jr. McConnell,Purchase,1001.0,R,Senate,1001 - 15000,NaN,49.059349,1001.0,15000.0,-1001.0,-15000.0,-3003.0,-45000.0,-24001.5,-305.752120,-20.403858
1,2018-06-13,2018-06-01,wfc,A. Mitchell Jr. McConnell,Purchase,1001.0,R,Senate,1001 - 15000,NaN,46.939678,1001.0,15000.0,-1001.0,-15000.0,-4004.0,-60000.0,-32002.0,-319.559072,-21.325242
2,2018-09-14,2018-09-06,wfc,A. Mitchell Jr. McConnell,Purchase,1001.0,R,Senate,1001 - 15000,NaN,50.084381,1001.0,15000.0,-1001.0,-15000.0,-5005.0,-75000.0,-40002.5,-299.494566,-19.986271
3,2019-03-06,2019-03-05,wfc,A. Mitchell Jr. McConnell,Purchase,1001.0,R,Senate,1001 - 15000,NaN,43.876400,1001.0,15000.0,-1001.0,-15000.0,-7007.0,-105000.0,-56003.5,-341.869433,-22.814087
4,2019-06-14,2019-06-03,vmc,A. Mitchell Jr. McConnell,Sale,250001.0,R,Senate,250001 - 500000,NaN,123.395256,250001.0,500000.0,250001.0,500000.0,241993.0,380000.0,310996.5,4052.019634,2026.017921


In [14]:
def calculate_variation(row):
    ticker = row['Ticker'].lower()
    transaction_date = row['TransactionDate']
    end_date = pd.to_datetime(transaction_date) + pd.DateOffset(months=3)

    try:
        # Create a Ticker object for the ticker symbol
        ticker_obj = yf.Ticker(ticker)
        
        # Retrieve historical stock price data for the specified date range
        stock_data = ticker_obj.history(start=transaction_date, end=end_date)
        
        # Check if the stock_data is empty (no data found)
        if stock_data.empty:
            return None  # Return None if no data is available
        else:
            # Calculate variation (you can adjust this calculation based on your specific requirements)
            variation = (stock_data['Close'].iloc[-1] - stock_data['Close'].iloc[0]) * row['Amount']
            return variation
    except Exception as e:
        print(f"Error for {ticker}: {e}")
        return None  # Return None in case of errors



In [17]:
import pandas as pd
from io import StringIO
import requests  # Import requests library

# Define a function to calculate the next three months' variation for a given transaction
def calculate_variation(row):
    ticker = row['Ticker']
    transaction_date = row['TransactionDate']
    end_date = pd.to_datetime(transaction_date) + pd.DateOffset(months=3)

    try:
        # Create the Yahoo Finance historical data URL
        url = f"https://query1.finance.yahoo.com/v7/finance/download/{ticker}"
        params = {
            'period1': int(pd.Timestamp(transaction_date).timestamp()),
            'period2': int(end_date.timestamp()),
            'interval': '1d',
        }
        
        # Send a GET request to retrieve historical stock price data
        response = requests.get(url, params=params)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Read the data into a DataFrame
            stock_data = pd.read_csv(StringIO(response.text))
            
            # Calculate variation (you can adjust this calculation based on your specific requirements)
            variation = (stock_data['Adj Close'].iloc[-1] - stock_data['Adj Close'].iloc[0]) * row['Amount']
            return variation
        else:
            print(f"Error for {ticker}: Request failed with status code {response.status_code}")
            return None  # Return None in case of errors
    except Exception as e:
        print(f"Error for {ticker}: {e}")
        return None  # Return None in case of errors




In [18]:
# Apply the calculate_variation function to each row in the DataFrame
df['Variation'] = df.apply(calculate_variation, axis=1)

# Print the resulting DataFrame with the calculated variation
print(df)

Error for wfc: Request failed with status code 403
Error for wfc: Request failed with status code 403
Error for wfc: Request failed with status code 403
Error for wfc: Request failed with status code 403
Error for vmc: Request failed with status code 403
Error for ir: Request failed with status code 403
Error for wfc: Request failed with status code 403
Error for wfc: Request failed with status code 403
Error for wfc: Request failed with status code 403
Error for wfc: Request failed with status code 403
Error for wfc: Request failed with status code 403
Error for wfc: Request failed with status code 403
Error for wfc: Request failed with status code 403
Error for wfc: Request failed with status code 403
Error for wfc: Request failed with status code 403
Error for wfc: Request failed with status code 403
Error for wfc: Request failed with status code 403
Error for kr: Request failed with status code 403
Error for wfc: Request failed with status code 403
Error for wfc: Request failed wit

KeyboardInterrupt: 